### Implementation of basic recommenders
#### Level 1
- Popularity-Based Recommender :This recommender suggests the most popular blogs across all users based on how many people interacted with them and how strongly they engaged.


In [8]:
import pandas as pd
import numpy as np

In [2]:
blogs_rating_df = pd.read_csv('data/Blog Ratings.csv')
blogs_df = pd.read_csv('data/Medium Blog Data.csv')

In [4]:
# Merge ratings with blog metadata
popular_df = blogs_rating_df.merge(blogs_df, on='blog_id', how='inner')



In [5]:
popular_df.head(3)

,blog_id,userId,ratings,author_id,blog_title,blog_content,blog_link,blog_img,topic,scrape_time
0,9025,11,3.5,5960,How I became a Frontend Developer,A little bit of background about me: as a teen...,https://medium.com/@steven.dornan93/how-i-beca...,https://miro.medium.com/v2/resize:fill:140:140...,web-development,2023-04-04 08:53:52
1,9320,11,5.0,6155,Writing an Algorithm to Calculate Article Read...,You have probably noticed a read-time number u...,https://medium.com/@dpericich/writing-an-algor...,https://miro.medium.com/v2/resize:fill:140:140...,web-development,2023-04-04 08:53:52
2,9246,11,3.5,6114,Diving into HTML and the Tools of the Trade,It’s been an incredible first week as a Bytewi...,https://medium.com/@muhammadnaeemtahir/diving-...,https://miro.medium.com/v2/resize:fill:140:140...,web-development,2023-04-04 08:53:52


### Popularity-Based Recommender

In [10]:
# STEP 2: Group by blog to compute rating stats
popularity_stats = popular_df.groupby(['blog_id', 'blog_title']).agg(
    avg_rating=('ratings', 'mean'),
    num_ratings=('ratings', 'count')
).reset_index()

# STEP 3: Compute weighted popularity score
# Formula: score = avg_rating * log(num_ratings + 1)
popularity_stats['weighted_score'] = (
    popularity_stats['avg_rating'] * np.log1p(popularity_stats['num_ratings'])
)

# STEP 4: Sort blogs by weighted score (you can also try num_ratings or avg_rating)
top_blogs = popularity_stats.sort_values(by='weighted_score', ascending=False)

# STEP 5: Display top 10 popular blogs
print(top_blogs.head(10))

      blog_id                                         blog_title  avg_rating  \
8535     8582  Top Mobile App Development Trends To Look Out ...    3.981132   
8636     8683       Deploying Docker Containers with Bind Mounts    3.811321   
9618     9668  10 Advanced Techniques Every Senior React Engi...    3.823529   
2653     2685  Backend with Spring & Kotlin: Spring Data with...    3.730769   
8328     8374           Ultimate CI/CD For Flutter Mobile Apps 🚀    3.987500   
8738     8785  Flutter Widget Lifecycle: Everything You Need ...    3.603448   
2697     2730                               ReactJS Notlarım — 2    3.636364   
9470     9519  Redis vs. Other Databases: An In-Depth Compari...    3.793478   
2925     2959                                       Why FastAPI?    3.793478   
8823     8870            Flutter Web — Runtime Docker variables.    3.577586   

      num_ratings  weighted_score  
8535           53       15.880672  
8636           53       15.203298  
9618       

### Topic-Based Recommender

In [13]:
topics = popular_df['topic'].unique()
print(topics)

['web-development' 'blockchain' 'ai' 'flutter' 'nlp' 'backend'
 'app-development' 'machine-learning' 'data-analysis'
 'information-security' 'android' 'security' 'image-processing'
 'Software-Development' 'deep-learning' 'Cryptocurrency' 'cloud-computing'
 'cloud-services' 'web3' 'cybersecurity' 'dev-ops' 'data-science'
 'backend-development']


In [23]:
def get_top_blogs_by_topic(popular_df, topic, top_n=5):

    # Filter for the selected topic
    topic_df = popular_df[popular_df['topic'] == topic]
    
    # Compute average rating and count per blog
    topic_stats = topic_df.groupby(['blog_id', 'blog_title']).agg(
        avg_rating=('ratings', 'mean'),
        num_ratings=('ratings', 'count')
    ).reset_index()

    # Add weighted score = avg_rating * log(num_ratings + 1)
    topic_stats['weighted_score'] = topic_stats['avg_rating'] * np.log1p(topic_stats['num_ratings'])

    # Sort by weighted score
    return topic_stats.sort_values(by='weighted_score', ascending=False).head(top_n)


In [24]:
for topic in topics:
    print('Topic: ', topic)
    print(get_top_blogs_by_topic(popular_df=popular_df, topic=topic, top_n=3))
    print('---------')

Topic:  web-development
    blog_id                                         blog_title  avg_rating  \
72     9093  How I Improve my Skills in Typescript tips for...    4.050000   
0      9021  All JavaScript and TypeScript Features of the ...    3.533333   
15     9036  Best ChatGpt Prompts to Use for Developers: En...    3.438776   

    num_ratings  weighted_score  
72           30       13.907648  
0            45       13.527866  
15           49       13.452569  
---------
Topic:  blockchain
     blog_id                                    blog_title  avg_rating  \
264     4137                Ternoa Joins Forces with Banxa    4.863636   
509     4382                    Bitcoin is Soaring in 2023    4.156250   
606     4480  Is SiSu Network the New Trend in Blockchain?    4.307692   

     num_ratings  weighted_score  
264           11       12.085682  
509           16       11.775543  
606           13       11.368247  
---------
Topic:  ai
     blog_id                            

In [36]:
# Sort by scrape_time
sorted_df = popular_df.sort_values(by='scrape_time', ascending=True)

# First (earliest)
first_scrape_time = sorted_df['scrape_time'].iloc[0]

# Last (latest)
last_scrape_time = sorted_df['scrape_time'].iloc[-1]

# Print results
print("First scrape time:", first_scrape_time)
print("Last scrape time:", last_scrape_time)


First scrape time: 2023-02-27 07:37:48
Last scrape time: 2023-04-04 08:53:52


In [39]:
def get_rule_based_recommendations(popular_df, min_rating=3.5, after_date='2023-02-27', top_n=10):

    
    # Convert scrape_time to datetime
    popular_df['scrape_time'] = pd.to_datetime(popular_df['scrape_time'])

    # Apply rules: recent + high rating
    filtered = popular_df[
        (popular_df['ratings'] >= min_rating) &
        (popular_df['scrape_time'] >= pd.to_datetime(after_date))
    ]
    
    # Rank by rating and recency
    recent_top = filtered.groupby(['blog_id', 'blog_title']).agg(
        avg_rating=('ratings', 'mean'),
        last_seen=('scrape_time', 'max'),
        num_ratings=('ratings', 'count')
    ).reset_index()

    return recent_top.sort_values(by=['avg_rating', 'last_seen'], ascending=[False, False]).head(top_n)

print(get_rule_based_recommendations(popular_df, min_rating=3.5, after_date='2023-03-01', top_n=15))


      blog_id                                         blog_title  avg_rating  \
2842     3097                             ChatGPT in Apple Notes         5.0   
2847     3102                   The Future Of Prompt Engineering         5.0   
2856     3111  Using AI in UX research: a structured and ethi...         5.0   
2857     3112                      ChatGPT — The Final Onslaught         5.0   
2859     3114   The AI hype bubble is the new crypto hype bubble         5.0   
2861     3116                     GPT-3 is going to be obsolete?         5.0   
2864     3119             6 Mind-Blowing AI Tools to Try in 2023         5.0   
2867     3122  Advancing AI to eliminate trial and error in m...         5.0   
2868     3123  The Powerful Writing Tool That’s Superior to C...         5.0   
2870     3125  Unleashing the Power of ChatGPT for Advanced W...         5.0   
2874     3129  WONDER DYNAMICS — Poised to Revolutionize Anim...         5.0   
2876     3131                  Understan